# TP3 : Algorithme de recherche de Grover

permet de parcourir une liste triee

In [ ]:
# Exécuter seulement dans Google Colab
!pip install myqlm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.2/833.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.5/648.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.0/661.0 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.0/911.0 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 742.5/742.5 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
   ━━━━

In [ ]:
import numpy as np

from qat.lang import QRoutine, H, CNOT, RY, Z, X, CCNOT, Program
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

# Description de l'algorithme

L'algorithme de Grover [https://fr.wikipedia.org/wiki/Algorithme_de_Grover], proposé en 1996 par Lov Grover, est l'algorithme de recherche quantique. Il consiste à rechercher un ou plusieurs éléments suivant un critère dans une liste d'éléments **non classés** de longueur $N=2^n$. Cet algorithme a pour intérêt de résoudre le problème de recherche en un temps en $O(\sqrt N)$ au lieu de $O(N)$ que proposerait une solution classique.

L'algorithme de Grover fonctionne de la manière suivante :
1) **Superposition** : On crée une superposition sur toutes les entrées possibles dans le registre quantique d'entrée $x$ à l'aide de portes $H$,
2) **Marquage** : Un **oracle** vient marquer les éléments qui vérifient le critère donné pour la recherche,
3) **Amplification** : On amplifie l'amplitude (donc la probabilité) des états marqués à l'aide de l'**opérateur de diffusion**.

Après plusieurs répétitions des étapes de marquage et d'amplification (en pratique le nombre d'itérations optimal est d'environ $\frac{\pi}{4}\sqrt{2^n}$), on peut extraire le résultat à l'aide d'une mesure. Ci-dessous est donné le circuit quantique annoté de l'algorithme de Grover

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/grover_circuit.png?raw=1" width=700/>
</center>

On peut visualiser l'évolution de l'algorithme en utilisant un histogramme d'amplitude comme ci-dessous. A partir de la superposition uniforme, avec l'état recherché (inconnu), on applique l'oracle qui va venir marquer cet état. Le marquage consiste à appliquer une phase de $-1$ sur l'état, son amplitude devient donc négative. Cela a pour effet de faire baisser la moyenne des amplitudes sur tous les états. Ensuite on applique l'opérateur de diffusion qui consiste à effectuer une symétrie par rapport à la moyenne des amplitudes des états. On a alors deux cas possibles pour la symétrie:

- L'état est un état non recherché : le symétrique par rapport à la moyenne fait baisser l'amplitude de cet état.
- L'état est un état recherché : le symétrique par rapport à la moyenne refait passer l'amplitude de cet état dans le positif en amplifiant sa valeur.

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/histogram_grover.png?raw=1" width=600/>
</center>

En une itération, on a commencé à amplifier l'amplitude de l'état recherché et en contre-partie fait diminuer celles des états non-recherchés. En itérant plusieurs fois avec les étapes de marquage et d'amplification on fini par grandement amplifier l'amplitude de l'état recherché et donc la probabilité de la mesurer en sortie.

# Implémentation de l'algorithme

Nous allons dans un premier temps implémenter chaque sous-bloc de l'algorithme de Grover pour ensuite les combiner.

## Superposition

**Question 1** : Implémenter la routine quantique qui permet de créer une superposition uniforme sur $n$ qubits


Concrètement, cela signifie qu'il faut appliquer une porte H sur chaque qubit de 0 à n−1.

In [4]:
#Surperposition sur n bit
def superposition(n):
    rout = QRoutine()
    for i in range(n):
        # Appliquer H sur le qubit i

        rout.apply(H, i)

    return rout

On vérifie que cela fonctionne bien sur $n=3$ qubits

In [5]:
circuit = superposition(3).to_circ()
display_result(circuit)

Etat |000>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |001>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |010>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |011>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |100>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |101>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |110>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |111>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)


<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

## Marquage

Pour la suite, nous allons implémenter deux oracles de marquage différents pour tester l'algorithme. Pour appeler des portes NOT multi-contrôlées il est possible d'utiliser l'instruction suivante :

<center>
    X.ctrl(nombre_de_qubits_de_controle)(qubits_de_ctrl, qubit_cible)
</center>

Par exemple, *X.ctrl(3)(q0,q1,q2,q3)* applique une porte $X$ sur le qubit q3, contrôlée par les qubits q0, q1 et q2.


L'objectif est à partir d'une fonction de marquage $f$ telle que

$$
f(x) = \begin{cases} 0 & \textrm{si x n'est pas recherché} \\ 1 & \textrm{si x est recherché} \end{cases}
$$

de créer un oracle $U$ tel que

$$
U(x)= (-1)^{f(x)}x.
$$

Pour ce faire, on va travailler en deux temps:
- Créer une routine quantique $U_f$ qui implémente la transformation $\ket{x}\ket{y} \rightarrow \ket{x}\ket{y \oplus f(x)}$
- Utiliser $U_f$ pour construire $U$ qui implémente la transformation $\ket{x} \rightarrow (-1)^{f(x)}\ket{x}$


### Fonction 1

Le circuit suivant permet d'appliquer le fonction 1, avec une entrée sur 4 qubits ($x$) et une sortie sur 1 qubit ($y$).

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/oracle_1.png?raw=1" width="300"/>
</center>

**Question 2** : A la main, retrouver les différentes valeurs de $x$ recherchées (pour lesquelles $y = 1$ en sortie)

Le qubit de sortie y **(initialisé à 0)** change de valeur (flip) à chaque fois qu'une porte est activée (c'est une somme modulo 2, ou XOR).

Pour avoir y=1 à la fin, il faut que le nombre total de "flips" soit impair car y commence à 0.

*   Porte de gauche s'active pour : (1,1,1,1)
*   Porte du milieu s'active pour : (1,0,0,1)
*   Porte de droite s'active pour : (0,1,1,0)

Combinaisons :
(1,0,0,1)
(1,1,0,1)
(1,0,1,1)
(1,1,1,1)

(0,1,1,0)
(0,1,1,1)
(1,1,1,0)

**Question 3** : Implémenter le circuit correspondant à la fonction 1 et vérifier que le circuit généré correspond bien à celui donné ci-dessus

In [7]:
def fonction_1():
    rout = QRoutine()

    # Porte de gauche : Contrôle sur 0,1,2,3 -> Cible q4
    rout.apply(X.ctrl(4), 0, 1, 2, 3, 4)

    # Porte du milieu : Contrôle sur 0 et 3 -> Cible q4
    # 2 contrôles
    rout.apply(X.ctrl(2), 0, 3, 4)

    # Porte de droite : Contrôle sur 1 et 2 -> Cible q4
    # On applique X avec 2 contrôles
    rout.apply(X.ctrl(2), 1, 2, 4)

    return rout

display_result(fonction_1())

Etat |00000>: probabilité 1.0, amplitude (1+0j)


**Question 4** : Verifier la réponse à la question 2 en simulant le circuit pour tous les valeurs possibles de $x$

In [11]:
#nb de qubits d'entrées
n = 4
nb_combinaisons = 2**n

for i in range(2**n):
    # Création du programme
    prog = Program()
    qubits = prog.qalloc(n + 1) # 4 qubits d'entrée (x) + 1 qubit de sortie (y)

    # Préparation de l'état d'entrée correspondant à i
    # On convertit i en binaire pour savoir où mettre des portes X
    # Ex: si i=9 (1001), on met des X sur q0 et q3 (si on lit de gauche à droite)
    # Pour faire simple, on va utiliser une astuce de conversion bit à bit
    bits_entree = []
    for j in range(n):
        # On vérifie si le j-ème bit de i est à 1
        if (i >> j) & 1:
            prog.apply(X, qubits[j])
            bits_entree.append(1)
        else:
            bits_entree.append(0)

    # Application de l'Oracle (Fonction 1)
    # On passe les qubits d'entrée (0 à 3) et le qubit de sortie (4)
    # Attention : fonction_1() renvoie une Routine, il faut apply
    prog.apply(fonction_1(), qubits)

    # Simulation
    # On veut juste voir si le qubit de sortie (indice n=4) est à 1
    circuit = prog.to_circ()
    result = qpu.submit(circuit.to_job())

    # On regarde l'état final.
    # result[0].state.int correspond à l'entier mesuré.
    # Si le qubit de sortie (le plus fort, bit 4) est à 1, l'état sera >= 2^4 = 16
    for sample in result:
        state_int = sample.state.int
        # On vérifie si le bit n (le 5ème bit, celui de sortie) est à 1
        if (state_int >> n) & 1:
            print(f"Entrée trouvée : {bits_entree} => y=1")



Entrée trouvée : [1, 0, 0, 0] => y=1
Entrée trouvée : [1, 1, 0, 0] => y=1
Entrée trouvée : [1, 0, 1, 0] => y=1
Entrée trouvée : [1, 1, 1, 0] => y=1
Entrée trouvée : [1, 0, 0, 1] => y=1
Entrée trouvée : [1, 1, 0, 1] => y=1
Entrée trouvée : [1, 0, 1, 1] => y=1
Entrée trouvée : [1, 1, 1, 1] => y=1


### Fonction 2

Le ciruit suivant permet d'appliquer la fonction 2.

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/oracle_2.png?raw=1" width="450"/>
</center>

**Question 5** : A la main, retrouver les différentes valeurs de $x$ recherchées (pour lesquelles $y=1$)

(0,0,0,0) X

(0,0,0,1) X

(0,0,1,0) X

(0,0,1,1) X

(0,1,0,0) X

(0,1,0,1) X

(0,1,1,0) V ← Porte 2 active (1 flip)

(0,1,1,1) X

(1,0,0,0) X

(1,0,0,1) V ← Porte 1 active (1 flip)

(1,0,1,0) X

(1,0,1,1) X

(1,1,0,0) X

(1,1,0,1) X

(1,1,1,0) X

(1,1,1,1) X




**Question 6** : Implémenter le circuit correspondant à la fonction 2 et vérifier que le cicruit généré correpsond bien à celui ci-dessus

In [12]:
def fonction_2():
    rout = QRoutine()

    # Porte 1 : Contrôle sur 0,3 -> Cible q4
    rout.apply(X.ctrl(2), 0, 3, 4)

    # Porte 2 : Contrôle sur 1 et 2 -> Cible q4
    # On applique X avec 2 contrôles
    rout.apply(X.ctrl(2), 1, 2, 4)

    # Porte 3 : Contrôle sur 0,1 et 2 -> Cible q4
    rout.apply(X.ctrl(3), 0, 1, 2, 4)

    # Porte 4 : Contrôle sur 0,1 et 3 -> Cible q4
    rout.apply(X.ctrl(3), 0, 1, 3, 4)

    # Porte 5 : Contrôle sur 0, 2 et 3 -> Cible q4
    rout.apply(X.ctrl(3), 0, 2, 3, 4)

    # Porte 5 : Contrôle sur 1, 2 et 3 -> Cible q4
    rout.apply(X.ctrl(3), 1, 2, 3, 4)

    return rout

**Question 7** : Vérifier la réponse à la question 5 en simulant le circuit ci-dessus pour toutes les valeurs possibles de $x$ en entrée

In [13]:
#TODO

Entrée trouvée : [1, 0, 0, 0] => y=1
Entrée trouvée : [1, 1, 0, 0] => y=1
Entrée trouvée : [1, 0, 1, 0] => y=1
Entrée trouvée : [1, 1, 1, 0] => y=1
Entrée trouvée : [1, 0, 0, 1] => y=1
Entrée trouvée : [1, 1, 0, 1] => y=1
Entrée trouvée : [1, 0, 1, 1] => y=1
Entrée trouvée : [1, 1, 1, 1] => y=1


<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

### Oracle de marquage

Maintenant que nous avons implémenté les fonctions $f1$ et $f2$ qui, pour rappel, appliquent la transformation suivante (pour différents cas de recherche)

$$
f(x) = \begin{cases} 0 & \textrm{si x n'est pas recherché} \\ 1 & \textrm{si x est recherché} \end{cases},
$$

nous devons implémenter un oracle $U$ tel que

$$
U(x)= (-1)^{f(x)}x.
$$

Formulé autrement, on veut appliquer une phase de $-1$ quand $f(x)=1$, donc que l'élément $x$ est recherché, sinon rien. On peut alors utilisé une porte quantique de base de l'informatique quantique qui transforme

- $\ket{0} \rightarrow \ket{0}$
- $\ket{1} \rightarrow -\ket{1}$.

La porte Z est adéquate pour effectuer cette transformation. L'idée pour créer l'oracle de marquage en utilisant une porte Z est la suivante

0) l'état initial est $\ket{x} \ket{y=0}$
1) On applique la fonction $f$ pour obtenir $\ket{x} \ket{f(x)}$
2) On applique une porte Z sur le qubit $y$, qui a pour effet de rajouter une phase sur les états marqués uniquement
3) On applique l'inverse de la fonction $f$ pour réinitialiser le qubit $y$ dans l'état 0

On a alors en sortie de l'oracle

- Appliquer une phase quand nécessaire pour marquer les états
- Un qubit ancillaire $y$ toujours à 0, que l'on peut ignorer pour la suite

Pour appeler l'inverse d'une routine quantique dans notre implémentation on peut utiliser la méthode *dag* comme suit :

<center>
    ma_routine.dag()(qubits)
</center>


**Question 8** : Implémenter la routine *oracle_marquage* en suivant les étapes détaillées ci-dessus

In [ ]:
def oracle_marquage(fonction):
    rout = QRoutine()

    return rout

**Question 9** : Vérifier à l'aide d'une simulation que l'oracle de marquage applique bien une phase de $-1$ sur les amplitudes des états marqués de la fonction 1 et de la fonction 2

In [ ]:
# Fonction 1

Etat |00000>: probabilité 1.0, amplitude (1+0j)
Etat |10000>: probabilité 1.0, amplitude (1+0j)
Etat |01000>: probabilité 1.0, amplitude (1+0j)
Etat |11000>: probabilité 1.0, amplitude (1+0j)
Etat |00100>: probabilité 1.0, amplitude (1+0j)
Etat |10100>: probabilité 1.0, amplitude (1+0j)
Etat |01100>: probabilité 1.0, amplitude (-1+0j)
Etat |11100>: probabilité 1.0, amplitude (-1+0j)
Etat |00010>: probabilité 1.0, amplitude (1+0j)
Etat |10010>: probabilité 1.0, amplitude (-1+0j)
Etat |01010>: probabilité 1.0, amplitude (1+0j)
Etat |11010>: probabilité 1.0, amplitude (-1+0j)
Etat |00110>: probabilité 1.0, amplitude (1+0j)
Etat |10110>: probabilité 1.0, amplitude (-1+0j)
Etat |01110>: probabilité 1.0, amplitude (-1+0j)
Etat |11110>: probabilité 1.0, amplitude (-1+0j)


In [ ]:
# Fonction 2

Etat |00000>: probabilité 1.0, amplitude (1+0j)
Etat |10000>: probabilité 1.0, amplitude (1+0j)
Etat |01000>: probabilité 1.0, amplitude (1+0j)
Etat |11000>: probabilité 1.0, amplitude (1+0j)
Etat |00100>: probabilité 1.0, amplitude (1+0j)
Etat |10100>: probabilité 1.0, amplitude (1+0j)
Etat |01100>: probabilité 1.0, amplitude (-1+0j)
Etat |11100>: probabilité 1.0, amplitude (1+0j)
Etat |00010>: probabilité 1.0, amplitude (1+0j)
Etat |10010>: probabilité 1.0, amplitude (-1+0j)
Etat |01010>: probabilité 1.0, amplitude (1+0j)
Etat |11010>: probabilité 1.0, amplitude (1+0j)
Etat |00110>: probabilité 1.0, amplitude (1+0j)
Etat |10110>: probabilité 1.0, amplitude (1+0j)
Etat |01110>: probabilité 1.0, amplitude (1+0j)
Etat |11110>: probabilité 1.0, amplitude (1+0j)


## Amplification

L'objectif est désormais d'amplifier les états marqués, à l'aide de l'opérateur de diffusion. L'opérateur de diffusion sur $n$ qubits correspond au circuit quantique suivant

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/circuit_diffusion.png?raw=1" width="450"/>
</center>

**Question 10** : Implémenter la routine quantique operateur de diffusion ayant pour paramètre $n$

In [ ]:
def operateur_de_diffusion(n):
    rout = QRoutine()

    return rout

## Algorithme de Grover

Maintenant que nous possédons toutes les routines quantiques nécessaires, nous pouvons les créer la routine de l'algorithme de Grover. Pour rappel, le circuit quantique correspondant à l'algorithme de Grover est le suivant

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP3_Grover/img/grover_circuit.png?raw=1" width=600/>
</center>

**Question 11**: Créer la routine quantique *grover* qui prend en entrée un *oracle* et un nombre de qubit *n*

In [ ]:
def grover(oracle, n):
    rout = QRoutine()

    return rout

Nous pouvons alors vérifier qu'on trouve bien en sortie de l'algorithme les états recherchés pour la fonction 1 et 2

In [ ]:
n = 4

oracle_1 = oracle_marquage(fonction_1())
circuit_1 = grover(oracle_1, n+1).to_circ()

display_result(circuit_1, idx=[0,1,2,3])

Etat |0000>: probabilité 0.001022875308990468, amplitude None
Etat |0001>: probabilité 0.001022875308990468, amplitude None
Etat |0010>: probabilité 0.0010228753089904664, amplitude None
Etat |0011>: probabilité 0.0010228753089904664, amplitude None
Etat |0100>: probabilité 0.0010228753089904633, amplitude None
Etat |0101>: probabilité 0.001022875308990466, amplitude None
Etat |0110>: probabilité 0.14154201745986833, amplitude None
Etat |0111>: probabilité 0.14154201745986833, amplitude None
Etat |1000>: probabilité 0.0010228753089904694, amplitude None
Etat |1001>: probabilité 0.14154201745986833, amplitude None
Etat |1010>: probabilité 0.0010228753089904783, amplitude None
Etat |1011>: probabilité 0.14154201745986839, amplitude None
Etat |1100>: probabilité 0.0010228753089904746, amplitude None
Etat |1101>: probabilité 0.14154201745986833, amplitude None
Etat |1110>: probabilité 0.14154201745986827, amplitude None
Etat |1111>: probabilité 0.14154201745986827, amplitude None


In [ ]:
n = 4

oracle_2 = oracle_marquage(fonction_2())
circuit_2 = grover(oracle_2, n+1).to_circ()

display_result(circuit_2, idx=[0,1,2,3])

Etat |0000>: probabilité 0.032287597656249716, amplitude None
Etat |0001>: probabilité 0.03228759765624976, amplitude None
Etat |0010>: probabilité 0.03228759765624973, amplitude None
Etat |0011>: probabilité 0.03228759765624974, amplitude None
Etat |0100>: probabilité 0.03228759765624973, amplitude None
Etat |0101>: probabilité 0.03228759765624974, amplitude None
Etat |0110>: probabilité 0.27398681640624784, amplitude None
Etat |0111>: probabilité 0.03228759765624973, amplitude None
Etat |1000>: probabilité 0.03228759765624973, amplitude None
Etat |1001>: probabilité 0.27398681640624784, amplitude None
Etat |1010>: probabilité 0.03228759765624974, amplitude None
Etat |1011>: probabilité 0.03228759765624973, amplitude None
Etat |1100>: probabilité 0.03228759765624974, amplitude None
Etat |1101>: probabilité 0.03228759765624973, amplitude None
Etat |1110>: probabilité 0.03228759765624976, amplitude None
Etat |1111>: probabilité 0.032287597656249716, amplitude None
